In [2]:
%matplotlib inline

In [3]:
from scipy import stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_context("poster")

# Basic Stats on the data

Focus:

* to understand each data element
* to get an idea of what type of algorithm needs to be built
    * Event level - simple line-by-line algorithm
    * User level - Needs to hold all user data and aggregate up for each event coming in

In [4]:
def descr(filen):
    f = pd.read_csv(filen)
    print f.head()
    print f.describe()

## destinations.csv

In [5]:
print len(pd.read_csv('data/destinations.csv'))

62106


In [6]:
descr('data/destinations.csv')

   srch_destination_id        d1        d2        d3        d4        d5  \
0                    0 -2.198657 -2.198657 -2.198657 -2.198657 -2.198657   
1                    1 -2.181690 -2.181690 -2.181690 -2.082564 -2.181690   
2                    2 -2.183490 -2.224164 -2.224164 -2.189562 -2.105819   
3                    3 -2.177409 -2.177409 -2.177409 -2.177409 -2.177409   
4                    4 -2.189562 -2.187783 -2.194008 -2.171153 -2.152303   

         d6        d7        d8        d9    ...         d140      d141  \
0 -1.897627 -2.198657 -2.198657 -1.897627    ...    -2.198657 -2.198657   
1 -2.165028 -2.181690 -2.181690 -2.031597    ...    -2.165028 -2.181690   
2 -2.075407 -2.224164 -2.118483 -2.140393    ...    -2.224164 -2.224164   
3 -2.115485 -2.177409 -2.177409 -2.177409    ...    -2.161081 -2.177409   
4 -2.056618 -2.194008 -2.194008 -2.145911    ...    -2.187356 -2.194008   

       d142      d143      d144      d145      d146      d147      d148  \
0 -2.198657 -2.19

In [8]:
### This is a pretty big piece of data to work with locally, lets sample it for the preliminary stuff 
pd.read_csv('data/test.csv').columns

        id            date_time  site_name  posa_continent  \
0  2104079  2015-11-05 23:20:56          2               3   
1   781110  2015-06-17 18:15:57          2               3   
2  1602093  2015-06-30 15:07:26          2               3   
3  2008933  2015-08-10 09:55:47          2               3   
4  1441610  2015-05-03 17:17:11         24               2   

   user_location_country  user_location_region  user_location_city  \
0                     66                   321               47535   
1                     66                   332               48650   
2                     66                   363                2495   
3                     57                   342                5021   
4                      3                    84                6148   

   orig_destination_distance  user_id  is_mobile      ...          srch_ci  \
0                  2308.2647   990284          0      ...       2016-02-12   
1                   833.8805   194649          0  

So from the description this is an event basis - How many users do we have?

Shall we be looking at an event based algorithm or a user algorithm?

## train.csv

In [ ]:
### This is a pretty big piece of data to work with locally, lets sample it for the preliminary stuff 
pd.read_csv('data/train.csv').sample(n=1000000).to_csv('data/train_sample.csv',index=False)
    
descr('data/train_sample.csv')

             date_time  site_name  posa_continent  user_location_country  \
0  2013-09-04 13:47:29          2               3                     66   
1  2013-10-13 12:14:03         11               3                    205   
2  2013-12-18 15:05:35         34               3                    205   
3  2014-11-21 11:30:57          8               4                     77   
4  2013-11-25 07:09:18         11               3                    205   

   user_location_region  user_location_city  orig_destination_distance  \
0                   321                3263                   262.2906   
1                   135               36086                   169.2580   
2                   354               43201                    72.3942   
3                   947               50710                 10831.4902   
4                   135               27655                  2377.6031   

   user_id  is_mobile  is_package      ...        srch_children_cnt  \
0   422363          0      

In [ ]:
ftest = pd.read_csv('data/train.csv')
print len(ftest), len(ftest['user_id'].unique()), ftest.groupby('user_id')['is_booking'].count().max()

In [ ]:
##Whats the distribution of user calls? do we need a recursive user algorithm or can we just simply rescore each event?
uftest = ftest.groupby('user_id')['is_booking'].count().reset_index()
uftest.columns = ['user_id','count']
uftest.head()

In [ ]:
plt.hist(uftest['count'],bins=530)
plt.ylabel("count of users")
plt.xlabel("No of events per user")
plt.title("No of events per user")
plt.show()

In [ ]:
## % of users who have 1 event and less than 10
print round(float(len(uftest[uftest['count']<= 10])) / float(len(uftest))*100,2)

So it looks like its going to need a user level algorithm as we have alot of users with multiple events - although 40% of users have less than 10 events

Lets split the analysis into two types:

1 Event level algorithm

2 User level algorithm

## test.csv

Lets just make sure we have a correct test csv, looking at the columns and size

In [43]:
print len(pd.read_csv('data/test.csv'))

2528243


In [44]:
print pd.read_csv('data/test.csv').columns

Index([u'id', u'date_time', u'site_name', u'posa_continent',
       u'user_location_country', u'user_location_region',
       u'user_location_city', u'orig_destination_distance', u'user_id',
       u'is_mobile', u'is_package', u'channel', u'srch_ci', u'srch_co',
       u'srch_adults_cnt', u'srch_children_cnt', u'srch_rm_cnt',
       u'srch_destination_id', u'srch_destination_type_id', u'hotel_continent',
       u'hotel_country', u'hotel_market'],
      dtype='object')


## Are we looking at new users or just existing users?

In [45]:
train_users = pd.read_csv('data/train.csv')['user_id']
test_users = train = pd.read_csv('data/test.csv')['user_id']

print test_users[~test_users.isin(train_users)].count()

0


So we are definitely loking at a user level algorithm rather than just an event based one.

Data manipulation next - will need to aggregate to user level

### Train and test checks

In [48]:
trains = pd.read_csv('data/train_sample.csv')
tests = pd.read_csv('data/test_sample.csv')

In [50]:
trains.columns

Index([u'date_time', u'site_name', u'posa_continent', u'user_location_country',
       u'user_location_region', u'user_location_city',
       u'orig_destination_distance', u'user_id', u'is_mobile', u'is_package',
       u'channel', u'srch_ci', u'srch_co', u'srch_adults_cnt',
       u'srch_children_cnt', u'srch_rm_cnt', u'srch_destination_id',
       u'srch_destination_type_id', u'is_booking', u'cnt', u'hotel_continent',
       u'hotel_country', u'hotel_market', u'hotel_cluster'],
      dtype='object')

In [51]:
tests.columns

Index([u'id', u'date_time', u'site_name', u'posa_continent',
       u'user_location_country', u'user_location_region',
       u'user_location_city', u'orig_destination_distance', u'user_id',
       u'is_mobile', u'is_package', u'channel', u'srch_ci', u'srch_co',
       u'srch_adults_cnt', u'srch_children_cnt', u'srch_rm_cnt',
       u'srch_destination_id', u'srch_destination_type_id', u'hotel_continent',
       u'hotel_country', u'hotel_market'],
      dtype='object')

In [49]:
print len(trains.columns),len(tests.columns)

24 22


So just the id and hotel cluster not there as expected

In [5]:
traint = pd.read_csv('data/train.csv')

traint.groupby('user_id')['hotel_cluster'].agg(lambda x: len(x.unique())).max()

88